In [1]:
%config Completer.use_jedi = False

In [29]:
from transformers import AutoModel,AutoConfig,AutoTokenizer
from torch import nn
import torch
import pandas as pd
from torch.utils.data import  dataloader
from transformers import AdamW
from sklearn.metrics import accuracy_score
from tqdm import  tqdm

In [3]:
def load_disdbert():
    config = AutoConfig.from_pretrained('/data/yuchen/projects/transformers_test/model/distilbert/config.json')
    model = AutoModel.from_config(config)
    # tokenizer = AutoTokenizer.from_pretrained('/data/yuchen/projects/test/transformers_test/model/distilbert')
    # inputs = tokenizer('hello world',return_tensors='pt')
    return model

class Classification(nn.Module):
    def __init__(self):
        super(Classification,self).__init__()
        self.distilbert = load_disdbert()
        self.ffn = nn.Linear(768,2)
        self.softmax = nn.Softmax(dim=1)
    def forward(self,inputs):
        cls = self.distilbert(**inputs)['last_hidden_state'][:,0,:]
        cls =  self.ffn(cls)
        # cls = self.softmax(cls)
        return cls

def loss(score,label):
    loss = nn.functional.cross_entropy(score,label)
    return loss

In [4]:
BATCH_SIZE=2
imdb = pd.read_csv('/data/yuchen/projects/transformers_test/data/IMDB Dataset.csv')
imdb.sentiment = imdb.sentiment.apply(lambda x: 0 if x == 'positive' else 1)#.astype(float)
imdb.review = imdb.review.astype(str)
data = list(zip(imdb.review.to_list(),imdb.sentiment.to_list()))

In [30]:
train_dl = dataloader.DataLoader(data[:-10000],batch_size=BATCH_SIZE,shuffle=True)
valid_dl = dataloader.DataLoader(data[-1000:],batch_size=BATCH_SIZE,shuffle=True)

In [31]:
model =  Classification().cuda()
tokenizer = AutoTokenizer.from_pretrained('/data/yuchen/projects/transformers_test/model/distilbert')

In [32]:
valid_result = []
for valid_data, label in tqdm(valid_dl):
    inputs = tokenizer(list(valid_data),return_tensors='pt',padding='max_length',max_length=512,truncation=True).to(torch.cuda.current_device())
    logits = model(inputs)
    pro = nn.functional.softmax(logits,dim=1)
    pre = pro.argmax(dim=1)
    pre = pre.detach().cpu().numpy()
    accuracy_score(label,pre)
    valid_result.append(accuracy_score(label,pre))
print(sum(valid_result) / len(valid_result))
    # break

100%|██████████| 500/500 [00:13<00:00, 36.93it/s]0.48

